<a href="https://colab.research.google.com/github/BHouwens/kitchen_sink/blob/main/Tabular.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Tabular Boilerplate Notebook**

Tabular modeling takes data in the form of a table (like a spreadsheet or CSV), where the objective is to predict the value in one column based on the values in the other columns. This notebook will serve as a boilerplate handler for tabular data modeling, with sections laid out for each major step of the modeling process.

**REMEMBER**: This boilerplate is just that: boilerplate! It's a good idea to perform your own exploration in a manner that's specific to your given dataset. The default boilerplate uses the Titanic dataset from Kaggle as an example.

## **Setup**

This is the section for setting up your work environment, with boilerplate setups for a number of mainstream options. Don't see one you like? Add your own!

In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

### **Imports and Installs**

Declare all your project's required imports and installs here:

In [ ]:
# Installs
!pip install -Uqq fastai waterfallcharts treeinterpreter dtreeviz

In [ ]:
# Imports
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG
import numpy as np
import math
import seaborn as sn
from scipy import stats
import xgboost as xgb

In [ ]:
# Statsmodels install (only necessary if module is needed and Colab gives trouble)
! pip install --upgrade Cython
! pip install --upgrade git+https://github.com/statsmodels/statsmodels

In [ ]:
# Set up some constraints here, if desired
pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

### **Colab Setup**

You can set up the Google Colab environment for data by mounting your Google Drive:

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

%cd gdrive/My Drive/



---



## **Data Collection**

Data specific to your current task can be collected here. There'll be different setups depending on where you're running this notebook, but the output here will be used for further data exploration.

### **Kaggle Datasets**

Get yourself started with some Kaggle datasets. First, set up your API credentials:

In [ ]:
creds = {"username":"USERNAME_HERE","key":"API_KEY_HERE"}

Then set the credentials up in the `kaggle.json` so that Kaggle knows where to look for them in API calls:

In [ ]:
!mkdir .kaggle
!mv .kaggle /root/
!touch /root/.kaggle/kaggle.json

!ls /root/.kaggle

Then write the credentials to `kaggle.json` with the correct permissions setup to enable access to Kaggle datasets:

In [ ]:
import json
import zipfile
import os

with open('/root/.kaggle/kaggle.json', 'w') as file:
    json.dump(creds, file)

!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!export KAGGLE_CONFIG_DIR=/root/.kaggle/kaggle.json

Before we finally install Kaggle itself:

In [ ]:
!pip install kaggle

from kaggle import api

Now let's fetch our Kaggle data:

In [ ]:
if not os.path.exists('data'):
  os.makedirs('data')

# Let's get the Titanic dataset
api.competition_download_files('titanic', path='data')

import zipfile
with zipfile.ZipFile("data/titanic.zip","r") as zip_ref:
    zip_ref.extractall("data")

os.remove('data/titanic.zip')
os.remove('data/gender_submission.csv')

In [ ]:
!ls data

Finally, let's set up some variables for taking the data exploration further:

In [ ]:
path = "data"
train_data = "train.csv"



---



## **Exploratory Data Analysis**

EDA can be performed here, where you'll find cells for showing batches, as well as utility functions for displaying certain analytics. It also contains headings to prompt some thinking about possible exploratory approaches. For tabular data in particular, this section will include tree classifiers for model/data fit inspection.

**REMEMBER**: This section is not prescriptive! Add and remove from it as you want and need to.

### **Look at the Data**

In [ ]:
# Setup the dataset path
train_path = "{b}/{d}".format(b=path, d=train_data)

# And read it in to a df
df = pd.read_csv(train_path, low_memory=False)

In [ ]:
# Set a "reset" df, in case we want to reset the data 
reset_df = df.copy(deep=True)

In [ ]:
print(df.columns)
len(df)

In [ ]:
# Some values will be individual specific and not useful in a test environment
df.drop(['PassengerId', 'Name', 'Ticket'], inplace=True, axis=1, errors='ignore')

We can a bit of a description of our dataset using `df.describe()`

In [ ]:
df.describe()

We can also view the interactions between all possible feature pairs. This isn't necessarily doable for larger datasets, but for the sake of our example it can be done for all features directly:

In [ ]:
# Create the default pairplot
sn.pairplot(df)

We can then set our dependent variable, the one we care about. We can also assign every other feature to an independent variablet set.

In [ ]:
dep_var = "Survived"
ind_var = [c for c in df.columns if c != dep_var]

It's always important to take a look at some of the entries themselves so that we can develop a better intuition for what we're working with.

In [ ]:
df.head(10)

We can see `NaN` values, which we'll need to decide how we want to handle, as well as pick out any potentially interesting features based on their values. It may also be useful to pick out "magic features", those which have a strong correlation to the target given some relatively simple transformation.

### **Handling Dates**

Dates often pose a challenge from an encoding point of view. They often have a lot of semantic meaning to us though (did it occur on a weekend? a holiday? etc), so we may want to encode special properties of the dates in the dataset.

In [ ]:
# Example from fast.ai Tabular core
date_df = pd.DataFrame({'date': ['2019-12-04', None, '2019-11-15', '2019-10-24']})
date_df = add_datepart(date_df, 'date')
date_df.head()

Further date-related features can be built if there is something specifically relevant to our project. For example, do we care about sales in a specific holiday season, or is a particular virus infection rate seasonal? 

### **Handling Rank in Ordinal Columns**

Some categorical data will be ranked (*ordinal columns*), and for these features it may be useful to tell Pandas about how these categories are ordered.

In [ ]:
# As an example, let's take the passenger class for the Titanic dataset
df['Pclass'].unique()

In [ ]:
# export 
def define_ordinal_rank(df, col, ranks):
  """
  Defines ordinal ranking of a column in a dataset

  Args:
    df: DataFrame to define for
    col: Column to define for
    ranks: An array of the ordinal ranking for col
  """
  df[col] = df[col].astype('category')
  df[col].cat.set_categories(ranks, ordered=True, inplace=True)

In [ ]:
define_ordinal_rank(df, 'Pclass', [1, 2, 3])

### **Handling Missing Values**

It's extremely common to find missing content in datasets and it's important to decide how to handle these. Some libraries, like fastai, do have built in handlers for this, but may be different from those found in something like Pandas.

In [ ]:
df = reset_df.copy(deep=True)

In [ ]:
# We can first find which columns contain NaN values
df.columns[df.isna().any()].tolist()

We can mark the existence of `NaN` in a row-column pair by creating a new column for each of the columns that contain `NaN`. In each of these new columns, if there was a `NaN` in that row we'll mark it with a 1 and if not we'll mark with a 0.

In [ ]:
# export
def mark_nan_cols(df):
  """
  Creates a new column for each NaN column in the df, 
  marking whether the row contained a NaN or not
  """
  nan_cols = df.columns[df.isna().any()].tolist()

  for col in nan_cols:
    df["{}_NaN".format(col)] = np.where(df[col].isnull(), 1, 0)

In [ ]:
mark_nan_cols(df)

In [ ]:
# Fill forward
df.fillna(method='ffill', inplace=True)

In [ ]:
# Fill with static value
df['Cabin'].fillna("D999", inplace=True)

In [ ]:
# Fill with mode
df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)

In [ ]:
# Fill with median
df['Age'].fillna(round(df['Age'].mean()), inplace=True)

In [ ]:
df.head(5)

In [ ]:
# Final check
df.columns[df.isna().any()].tolist()

### **Automated Data Checks and Processing**

Some simple checks can be made on the dataset automatically, depending on what it is you're looking for. In addition, certain types of augmentation or processing can be performed on the data in an automated fashion.

#### **Roulette Target**

A "roulette" occurs when there are duplicate rows with different target values. This makes training much more difficult, as target values may be close to random.

To start, we'll need to define some kind of acceptable amount of duplicates within the system. This should be generally okay as it's possible that some external, non-codified features have an effect on the target variable. So first we define an acceptable proportion of the dataset as duplicates:

In [ ]:
ACCEPTANCE_THRES = .02

And then we have two options. One is to do a simple check on the proportion of duplicates in the dataset and match it against our accepted proportion threshold. We do not consider rows that are full duplicates (that is they duplicate both the row values and the targets) so let's first set up a function to find the relevant duplicates:

In [ ]:
# export
def get_relevant_duplicates(df, dep_var):
  """
  Gets number of duplicates with differing targets
  """
  ind_var = [c for c in df.columns if c != dep_var]

  # We need to sift out full duplicate rows
  poss_dups = df.duplicated(ind_var).sum()
  full_dups = df.duplicated().sum()
  return poss_dups - full_dups

And then we can implement our simple solution:

In [ ]:
# Naive, workable solution
relevant_dups = get_relevant_duplicates(df, dep_var)
is_roulette = (relevant_dups / len(df) * 100) > ACCEPTANCE_THRES

print("Dataset is a roulette:", is_roulette)

This approach can work. A more rigorous approach is to apply a statistical hypothesis test against the accepted threshold and see if that holds up!

For this simple test we claim that the dataset is not a roulette ($H_0$) and perform a [1-proportion test](https://www.tutorialspoint.com/statistics/one_proportion_z_test.htm) to find the associated p-value for this hypothesis. Our alternative hypothesis is that our dataset contains fewer relevant duplicates than our accepted threshold:

In [ ]:
# export
import statsmodels.api as sm

ALPHA = .05

def roulette_test(df, dep_var, threshold):
  """
  Performs a 1-proportion z-test on df to check for roulette. 
  Null hypothesis is that the number of duplicates do not 
  constitute a roulette dataset, in that the number is lower than 
  the acceptance threshold
  """
  relevant_dups = get_relevant_duplicates(df, dep_var)
  if relevant_dups == 0: return False

  _, p_val = sm.stats.proportions_ztest(
      relevant_dups, len(df), len(df)*threshold, 'smaller')

  return p_val > ALPHA

In [ ]:
is_roulette = roulette_test(df, dep_var, ACCEPTANCE_THRES)
print("Dataset is a roulette:", is_roulette)

This check may or may not be relevant depending on the context of the dataset. Consider an actual roulette wheel's results or the results of a series of horse races, which may in fact contain a number of full duplicates that is higher than our threshold. In such a scenario, it would be beneficial to know before pursuing such a data science project further, as the dataset's value entropy may be too high to warrant further work.

#### **Highly Correlated Features**

Some features within the dataset may be highly correlated, and we may want to check for these and then make some decision about them.

In [ ]:
import seaborn as sn

corrMatrix = df.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

### **Feature Engineering**

Here we can do some feature engineering in order to improve the model's eventual understanding of the data.

In [ ]:
# We can start by creating some interactions
df['Age_Fare'] = df.apply(lambda row: row.Age * row.Fare, axis=1)
df['Age_Fare']

We may also need to specify interesting interactions based on the specifics of our datasets

In [ ]:
def is_female_and_high_class(row):
  if row.Sex == "female" and row.Pclass > 2:
    return 1
  return 0
 
df['Fem_HC'] = df.apply(is_female_and_high_class, axis=1)
df['Fem_HC'].head(10)

In [ ]:
def has_no_cabin_and_male(row):
  if row.Sex == "male" and row.Cabin_NaN == 1:
    return 1
  return 0

df['Mal_CNaN'] = df.apply(has_no_cabin_and_male, axis=1)
df['Mal_CNaN'].head()

#### **Target Encoding**

We can also do some target encoding for categorical inputs. This won't always be a worthwhile approach, but it's worth trying out.

In [ ]:
def target_encoding(df, feature, target=dep_var, agg_functions={"mean","std"}):
    agg=df.groupby(feature)[target].agg(agg_functions)
    agg.columns=[column+"_per_{}_{}".format(feature,target) for column in agg.columns.tolist()]
    return agg

This is only useful if we use a subset of the training set to try and encode features, because if we try to use the entire training set we will introduce massive data leakage.

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, random_state=42, test_size=.2)
train_VTE, val_VTE = train_test_split(train,random_state=42,test_size=0.1)
test_VTE = test.copy(deep=True)

categorical_features = ['Pclass', 'Sex', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Embarked', 'Age_NaN', 'Cabin_NaN', 'Fem_HC']

def add_target_encoding_features_validation(train, val, test):
    for feature in categorical_features: 
        agg = target_encoding(train, feature)
        train = train.merge(agg, how="left", on=feature)
        val = val.merge(agg, how="left", on=feature)
        test = test.merge(agg, how="left", on=feature)
        
    return train, val, test

train_VTE, val_VTE, test_VTE = add_target_encoding_features_validation(train_VTE, val_VTE, test_VTE)

df = pd.concat([train_VTE, val_VTE, test_VTE])
df.reset_index(inplace=True)
df.head(10)


### **Basic Data Cleaning**

#### **Removing Outliers**

In [ ]:
# export
def remove_outliers(df, cols=None):
  """
  Removes outlier entries in df 
  """
  if cols:
    w_df = df[cols]
  else:
    w_df = df

  z_scores = np.abs(stats.zscore(w_df, nan_policy='omit'))

  if cols:
    df = pd.concat([df, w_df], axis=1)

  print('z scores', print(len(np.where(z_scores > 3)[0])))
  df = df[(z_scores < 3).all(axis=1)]

  return df

#### **Drop Rare Features**

Certain features may have too few events to provide any meaningful insight to a model, and we can remove these automatically if needed.

In [ ]:
# export
def drop_rare_features(df, thres=.9):
  """
  Drops features in the dataset that have 
  events that are too rare to be statistically useful
  """
  rare_f = []
  print("Running with threshold", thres)
  print("")

  for col in df.columns:
    if df[col].name not in df.select_dtypes(include='category').columns:
      freq = df[col].value_counts(normalize=True)
      sum_freq = df[col].sum()
              
      # should be enough to check whether the most freq is dominant
      if freq.iloc[0] >= thres:
        rare_f.append(col)
        print("\x1b[31m{c}: {v}\x1b[0m".format(c=col, v=freq.iloc[0]))
      else:
        print("{c}: {v}".format(c=col, v=freq.iloc[0]))
  
  df = df.drop(rare_f, axis=1)
  return df

#### **Final Clean Up**

In [ ]:
# Remove outliers
print("DF before:", df.shape)
print("")

df = remove_outliers(df, ['Age_Fare','Fare'])

print("")
print("DF after removing outliers", df.shape)

In [ ]:
# Drop rare features
print("DF before:", len(df.columns))
print("")

df = drop_rare_features(df)

print("")
print("DF after drop rare features:", len(df.columns))

In [ ]:
print("DF before:", len(df))

ind_vars = [c for c in df.columns if c != dep_var]

# Remove duplicates
df.drop_duplicates(subset=ind_vars, inplace=True)
print("DF after drop duplicates:", len(df))

In [ ]:
df.drop(['index', 'Ticket'], inplace=True, axis=1, errors="ignore")

### **Conversion to TabularPandas**

If we want to convert our dataset into a `TabularPandas` dataset from [fast.ai](https://docs.fast.ai/tabular.core.html#TabularPandas) we can do so here, specifying all the preprocessing and splitting we'd require.

In [ ]:
# Start by defining procs
procs = [Categorify, Normalize]

#### **Splitting**

If we have timeseries data we'll probably want to manually specify a validation set, as the sequential nature may be important to the understanding of the data:

In [ ]:
# Example splitting for timeseries
cond = (df.saleYear<2011) | (df.saleMonth<10)
train_idx = np.where( cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx),list(valid_idx))

Alternatively you can split randomly:

In [ ]:
splits = RandomSplitter()(range_of(df))
splits

#### **Final Setup**

For categorical data we'll want to tell `TabularPandas` which columns are categorical and which are continuous:

In [ ]:
cont,cat = cont_cat_split(df, dep_var=dep_var)
print("Continuous columns:", cont)
print("Categorical columns:", cat)

In [ ]:
# Finally we set the TabularPandas df up
tdf = TabularPandas(df, procs, cat, cont, y_names=dep_var, splits=splits)

In [ ]:
tdf.show(5)

In [ ]:
tdf.items.head(5)

In [ ]:
len(tdf.train),len(tdf.valid)

We can also now save the data, as we've probably done a lot of work on it up to this point.

In [ ]:
save_pickle("{}/tdf.pkl".format(path),tdf)



---



## **Model**

Model work can be performed here, with utilities to help with cross-validation and architecture construction.

In [ ]:
# Let's start by setting up our separate sets
train_x, train_y = tdf.train.xs, tdf.train.y
valid_x, valid_y = tdf.valid.xs, tdf.valid.y

In [ ]:
print("First train features:", train_x.iloc[0])
print("")
print("First train label:", train_y.iloc[0])

### **Baseline Tree Model**

Tabular data has not been completely dominated by neural nets, and tree models are still a viable option to model such datasets. Even if we do go with a neural net (or some ensemble) for the final model, we can still infer a lot about how a model sees the data by starting with a tree.

In [ ]:
m = DecisionTreeClassifier(max_leaf_nodes=4)
m.fit(train_x, train_y);

We can then view the results visually:

In [ ]:
tree.plot_tree(m, feature_names=train_x.columns, precision=2)

In [ ]:
# DTreeViz result
samp_idx = np.random.permutation(len(train_y))[:500]
dtreeviz(m, train_x.iloc[samp_idx], train_y.iloc[samp_idx], train_x.columns, dep_var,
        fontname='DejaVu Sans', scale=1.6, label_fontsize=10,
        orientation='LR')

In [ ]:
m = RandomForestClassifier(n_estimators=50)
m.fit(train_x, train_y)

preds = m.predict(valid_x)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(valid_y, preds)

In [ ]:
# Let's find the reasonable number of trees
m = DecisionTreeClassifier()
m.fit(train_x, train_y);

plt.plot([accuracy_score(m.predict(train_x[:i+1]), valid_y[:i+1]) for i in range(150)]);

### **Tree Variance for Prediction Confidence**

How can we know the confidence of our random forest estimates? One simple way is to use the standard deviation of predictions across the trees, instead of just the mean. This tells us the relative confidence of predictions. In general, we would want to be more cautious of using the results for rows where trees give very different results (higher standard deviations), compared to cases where they are more consistent (lower standard deviations).

In [ ]:
m = RandomForestClassifier(n_estimators=50)
m.fit(train_x, train_y)

# Let's get predictions for each tree
preds = np.stack([t.predict(valid_x) for t in m.estimators_])
preds.shape

In [ ]:
# Now let's look at the standard deviation
preds_std = preds.std(0)

preds_std[:5]

As you can see, the confidence in the predictions varies widely and it seems like the trees are not able to come to a general consensus. For some passengers, there is a low standard deviation because the trees agree. For others it's higher, as the trees don't agree. This is information that would be useful in a production setting; for instance, if you were using a model to decide what items to bid on at auction, a low-confidence prediction might cause you to look more carefully at an item before you made a bid.

### **Feature Importance**

It's critical for us to know how the trees are making their decisions, and which features they're leaning on to do so. We can get these directly from `sklearn`'s random forest by looking in the `feature_importances_` attribute. Here's a simple function we can use to pop them into a DataFrame and sort them:

In [ ]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)

In [ ]:
# Let's view the most important features to our model
fi = rf_feat_importance(m, train_x)
fi[:10]

We can also plot it

In [ ]:
fi[:30].plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

We can remove the features that are of very low importance and see if that has any effect on the model.

In [ ]:
to_keep = fi[fi.imp>0.005].cols
list(to_keep)

In [ ]:
xs_imp = train_x[to_keep]
valid_xs_imp = valid_x[to_keep]

# Retrain
m = RandomForestClassifier(n_estimators=50)
m.fit(xs_imp, train_y);

In [ ]:
preds = m.predict(valid_xs_imp)
accuracy_score(valid_y, preds)

### **Final Model**

In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
dls.classes

In [ ]:
import torch.nn.functional as F

dls = tdf.dataloaders()
learn = tabular_learner(dls, metrics=accuracy, loss_func=F.cross_entropy)

In [ ]:
learn.lr_find()

In [ ]:
learn.fit_one_cycle(5, 1e-2)

In [ ]:
preds,targs = learn.get_preds()
preds



---



## **Inference and Deployment**

Here the model can finally be put to use, as well as exported for deployment in an external application.



---



## **Exports and Clean Up**

Here you can export any cells with the `#export` comment using `notebook2script.py`, as well as cleaning up any environmental changes such as data downloads to a cloud drive.

### **Exports**

In [ ]:
!python notebook2script.py Tabular.ipynb

### **Clean Up**

In [ ]:
# Tear down the data folder
!rm -rf data
!ls